# 🤝 Ensemble XGBoost + CatBoost

## 📋 Objetivos
- Combinar predicciones OOF de XGBoost y CatBoost
- Optimizar pesos de ensemble para mejor MAP@3
- Evaluar mejora de rendimiento vs modelos individuales
- Generar submission final del ensemble

## 🎯 Estrategia de Ensemble
- **XGBoost**: Variables categóricas codificadas + 19 features engineered
- **CatBoost**: Variables categóricas nativas + feature engineering adaptado
- **Ensemble**: Weighted averaging + voting strategies
- **Diversidad**: Diferentes representaciones categóricas maximizan diversidad

# 📦 Instalación y Configuración

In [ ]:
import pandas as pd
import numpy as np
import warnings
import json
import pickle
import glob
from pathlib import Path
from datetime import datetime
import sys
import os

# Imports de ML
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.optimize import minimize
from scipy.stats import mode

# Imports personalizados
sys.path.append('../src')
from metrics import map_at_k, calculate_map_score

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Configuración de paths
MODELS_PATH = Path('../models')
XGB_PATH = MODELS_PATH / 'XGB'
CATBOOST_PATH = MODELS_PATH / 'CatBoost'
ENSEMBLE_PATH = MODELS_PATH / 'Ensemble'
ENSEMBLE_PATH.mkdir(parents=True, exist_ok=True)

print("✅ Librerías importadas exitosamente")
print(f"📁 Directorio ensemble: {ENSEMBLE_PATH}")

# 🔍 Buscar Mejores Modelos

In [ ]:
def find_best_model(models_path, model_type):
    """
    Encuentra el mejor modelo basado en MAP@3 score en el nombre del directorio
    """
    model_dirs = [d for d in models_path.iterdir() if d.is_dir()]
    
    if not model_dirs:
        print(f"❌ No se encontraron modelos en {models_path}")
        return None
    
    best_score = 0
    best_dir = None
    
    print(f"\n🔍 Buscando mejor modelo {model_type}:")
    for model_dir in model_dirs:
        # Extraer score del nombre del directorio (formato: ModelType_MAP@3-XXXXX)
        dir_name = model_dir.name
        if 'MAP@3-' in dir_name or 'MAP3-' in dir_name:
            try:
                # Extraer score del nombre
                score_part = dir_name.split('MAP@3-')[-1] if 'MAP@3-' in dir_name else dir_name.split('MAP3-')[-1]
                score = float('0.' + score_part) if len(score_part) == 5 else float(score_part)
                print(f"  📊 {dir_name}: MAP@3 = {score:.5f}")
                
                if score > best_score:
                    best_score = score
                    best_dir = model_dir
            except (ValueError, IndexError):
                # Si no se puede extraer el score, buscar en metrics.json
                metrics_file = model_dir / f"{dir_name}_metrics.json"
                if metrics_file.exists():
                    try:
                        with open(metrics_file, 'r') as f:
                            metrics = json.load(f)
                        score = metrics.get('oof_results', {}).get('oof_map3', 0)
                        print(f"  📊 {dir_name}: MAP@3 = {score:.5f} (from metrics)")
                        
                        if score > best_score:
                            best_score = score
                            best_dir = model_dir
                    except:
                        print(f"  ❌ No se pudo leer métricas de {dir_name}")
    
    if best_dir:
        print(f"\n🏆 Mejor modelo {model_type}: {best_dir.name}")
        print(f"   📊 MAP@3: {best_score:.5f}")
        return best_dir, best_score
    else:
        print(f"❌ No se encontró modelo válido para {model_type}")
        return None, 0

# Buscar mejores modelos
best_xgb_dir, xgb_score = find_best_model(XGB_PATH, "XGBoost")
best_catboost_dir, catboost_score = find_best_model(CATBOOST_PATH, "CatBoost")

if not best_xgb_dir or not best_catboost_dir:
    print("\n❌ Error: No se encontraron ambos modelos necesarios para el ensemble")
    print("Asegúrate de haber entrenado tanto XGBoost como CatBoost primero.")
else:
    print(f"\n✅ Modelos encontrados para ensemble:")
    print(f"  🚀 XGBoost: {best_xgb_dir.name} (MAP@3: {xgb_score:.5f})")
    print(f"  🐱 CatBoost: {best_catboost_dir.name} (MAP@3: {catboost_score:.5f})")

# 📊 Cargar Predicciones OOF

In [ ]:
def load_oof_predictions(model_dir, model_name):
    """
    Cargar predicciones OOF de un modelo
    """
    # Buscar archivo OOF
    oof_files = list(model_dir.glob("*_oof_predictions.csv"))
    
    if not oof_files:
        print(f"❌ No se encontró archivo OOF para {model_name} en {model_dir}")
        return None
    
    oof_file = oof_files[0]
    print(f"📊 Cargando OOF de {model_name}: {oof_file.name}")
    
    oof_df = pd.read_csv(oof_file)
    print(f"   Forma: {oof_df.shape}")
    print(f"   Columnas: {oof_df.columns.tolist()}")
    
    return oof_df

def load_submission_predictions(model_dir, model_name):
    """
    Cargar predicciones de submission de un modelo
    """
    # Buscar archivo submission
    submission_files = list(model_dir.glob("*_submission.csv"))
    
    if not submission_files:
        print(f"❌ No se encontró archivo submission para {model_name} en {model_dir}")
        return None
    
    submission_file = submission_files[0]
    print(f"📤 Cargando submission de {model_name}: {submission_file.name}")
    
    submission_df = pd.read_csv(submission_file)
    print(f"   Forma: {submission_df.shape}")
    
    return submission_df

# Cargar predicciones OOF
if best_xgb_dir and best_catboost_dir:
    print("\n📊 Cargando predicciones Out-of-Fold...")
    
    xgb_oof = load_oof_predictions(best_xgb_dir, "XGBoost")
    catboost_oof = load_oof_predictions(best_catboost_dir, "CatBoost")
    
    print("\n📤 Cargando predicciones de test...")
    
    xgb_submission = load_submission_predictions(best_xgb_dir, "XGBoost")
    catboost_submission = load_submission_predictions(best_catboost_dir, "CatBoost")
    
    if xgb_oof is not None and catboost_oof is not None:
        print("\n✅ Todas las predicciones cargadas exitosamente")
    else:
        print("\n❌ Error cargando predicciones")

# 🔍 Análisis de Predicciones OOF

In [ ]:
if xgb_oof is not None and catboost_oof is not None:
    print("🔍 ANÁLISIS DE PREDICCIONES OOF")
    print("="*50)
    
    # Verificar que tengan el mismo número de muestras
    print(f"\n📊 Tamaños de OOF:")
    print(f"  XGBoost: {len(xgb_oof):,} muestras")
    print(f"  CatBoost: {len(catboost_oof):,} muestras")
    
    if len(xgb_oof) != len(catboost_oof):
        print("❌ Error: Los OOF tienen diferentes tamaños")
    else:
        # Analizar concordancia de etiquetas verdaderas
        if 'true_label' in xgb_oof.columns and 'true_label' in catboost_oof.columns:
            concordance = (xgb_oof['true_label'] == catboost_oof['true_label']).mean()
            print(f"\n✅ Concordancia etiquetas verdaderas: {concordance:.3%}")
            
            # Usar etiquetas del primer modelo como referencia
            true_labels = xgb_oof['true_label']
            
            # Extraer predicciones
            xgb_preds = xgb_oof['oof_prediction']
            catboost_preds = catboost_oof['oof_prediction']
            
            # Calcular métricas individuales
            xgb_acc = accuracy_score(true_labels, xgb_preds)
            catboost_acc = accuracy_score(true_labels, catboost_preds)
            
            print(f"\n📊 Rendimiento individual:")
            print(f"  XGBoost Accuracy: {xgb_acc:.6f}")
            print(f"  CatBoost Accuracy: {catboost_acc:.6f}")
            
            # Analizar concordancia de predicciones
            pred_concordance = (xgb_preds == catboost_preds).mean()
            print(f"\n🤝 Concordancia predicciones: {pred_concordance:.3%}")
            
            # Casos donde difieren
            different_preds = xgb_preds != catboost_preds
            print(f"🔄 Predicciones diferentes: {different_preds.sum():,} ({different_preds.mean():.1%})")
            
            # Analizar cuándo cada modelo es correcto
            xgb_correct = xgb_preds == true_labels
            catboost_correct = catboost_preds == true_labels
            
            both_correct = xgb_correct & catboost_correct
            only_xgb_correct = xgb_correct & ~catboost_correct
            only_catboost_correct = ~xgb_correct & catboost_correct
            both_wrong = ~xgb_correct & ~catboost_correct
            
            print(f"\n🎯 Análisis de aciertos:")
            print(f"  Ambos correctos: {both_correct.sum():,} ({both_correct.mean():.1%})")
            print(f"  Solo XGBoost correcto: {only_xgb_correct.sum():,} ({only_xgb_correct.mean():.1%})")
            print(f"  Solo CatBoost correcto: {only_catboost_correct.sum():,} ({only_catboost_correct.mean():.1%})")
            print(f"  Ambos incorrectos: {both_wrong.sum():,} ({both_wrong.mean():.1%})")
            
            # Potencial de mejora con ensemble
            potential_improvement = only_xgb_correct.sum() + only_catboost_correct.sum()
            print(f"\n🚀 Potencial mejora ensemble: {potential_improvement:,} casos ({potential_improvement/len(true_labels):.1%})")
        
        else:
            print("❌ No se encontraron columnas 'true_label' en los OOF")

# 🎯 Estrategias de Ensemble

In [ ]:
def simple_voting_ensemble(xgb_preds, catboost_preds):
    """
    Ensemble simple por votación mayoritaria
    """
    # Crear array con ambas predicciones
    all_preds = np.column_stack([xgb_preds, catboost_preds])
    
    # Votación por mayoría (en caso de empate, tomar XGBoost)
    ensemble_preds = []
    for i in range(len(all_preds)):
        preds_row = all_preds[i]
        # Si son iguales, usar cualquiera
        if preds_row[0] == preds_row[1]:
            ensemble_preds.append(preds_row[0])
        else:
            # En caso de desempate, usar XGBoost (generalmente más estable)
            ensemble_preds.append(preds_row[0])
    
    return np.array(ensemble_preds)

def weighted_ensemble(xgb_preds, catboost_preds, xgb_weight=0.5):
    """
    Ensemble ponderado simple (para predicciones categóricas)
    """
    # Para predicciones categóricas, usar votación ponderada por performance
    ensemble_preds = []
    
    for i in range(len(xgb_preds)):
        if xgb_preds[i] == catboost_preds[i]:
            # Si coinciden, usar esa predicción
            ensemble_preds.append(xgb_preds[i])
        else:
            # Si difieren, usar el modelo con mayor peso
            if xgb_weight >= 0.5:
                ensemble_preds.append(xgb_preds[i])
            else:
                ensemble_preds.append(catboost_preds[i])
    
    return np.array(ensemble_preds)

def confidence_based_ensemble(xgb_preds, catboost_preds, xgb_acc, catboost_acc):
    """
    Ensemble basado en confianza (accuracy) de cada modelo
    """
    # Calcular peso basado en accuracy
    total_acc = xgb_acc + catboost_acc
    xgb_weight = xgb_acc / total_acc
    
    return weighted_ensemble(xgb_preds, catboost_preds, xgb_weight)

# Evaluar diferentes estrategias de ensemble
if xgb_oof is not None and catboost_oof is not None and 'true_label' in xgb_oof.columns:
    print("\n🎯 EVALUANDO ESTRATEGIAS DE ENSEMBLE")
    print("="*50)
    
    true_labels = xgb_oof['true_label']
    xgb_preds = xgb_oof['oof_prediction']
    catboost_preds = catboost_oof['oof_prediction']
    
    # Accuracy individuales
    xgb_acc = accuracy_score(true_labels, xgb_preds)
    catboost_acc = accuracy_score(true_labels, catboost_preds)
    
    # 1. Simple Voting
    voting_preds = simple_voting_ensemble(xgb_preds, catboost_preds)
    voting_acc = accuracy_score(true_labels, voting_preds)
    
    # 2. Weighted by performance
    confidence_preds = confidence_based_ensemble(xgb_preds, catboost_preds, xgb_acc, catboost_acc)
    confidence_acc = accuracy_score(true_labels, confidence_preds)
    
    # 3. Diferentes pesos
    weights_to_test = [0.3, 0.4, 0.5, 0.6, 0.7]
    weighted_results = []
    
    for weight in weights_to_test:
        weighted_preds = weighted_ensemble(xgb_preds, catboost_preds, weight)
        weighted_acc = accuracy_score(true_labels, weighted_preds)
        weighted_results.append((weight, weighted_acc))
    
    # Mostrar resultados
    print(f"\n📊 RESULTADOS ACCURACY:")
    print(f"  XGBoost individual: {xgb_acc:.6f}")
    print(f"  CatBoost individual: {catboost_acc:.6f}")
    print(f"  Simple Voting: {voting_acc:.6f}")
    print(f"  Confidence-based: {confidence_acc:.6f}")
    
    print(f"\n⚖️ Pesos ponderados:")
    for weight, acc in weighted_results:
        print(f"  XGB weight {weight:.1f}: {acc:.6f}")
    
    # Encontrar mejor estrategia
    all_strategies = [
        ("XGBoost", xgb_acc, xgb_preds),
        ("CatBoost", catboost_acc, catboost_preds),
        ("Simple Voting", voting_acc, voting_preds),
        ("Confidence-based", confidence_acc, confidence_preds)
    ]
    
    # Agregar estrategias ponderadas
    for weight, acc in weighted_results:
        preds = weighted_ensemble(xgb_preds, catboost_preds, weight)
        all_strategies.append((f"Weighted XGB={weight:.1f}", acc, preds))
    
    # Ordenar por accuracy
    all_strategies.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n🏆 RANKING DE ESTRATEGIAS:")
    for i, (name, acc, _) in enumerate(all_strategies[:10]):
        print(f"  {i+1:2d}. {name:<20} {acc:.6f}")
    
    # Seleccionar mejor estrategia
    best_strategy_name, best_accuracy, best_ensemble_preds = all_strategies[0]
    
    print(f"\n🎯 MEJOR ESTRATEGIA: {best_strategy_name}")
    print(f"   Accuracy: {best_accuracy:.6f}")
    
    # Mejora vs modelos individuales
    improvement_vs_xgb = best_accuracy - xgb_acc
    improvement_vs_catboost = best_accuracy - catboost_acc
    improvement_vs_best_individual = best_accuracy - max(xgb_acc, catboost_acc)
    
    print(f"\n📈 MEJORAS:")
    print(f"   vs XGBoost: {improvement_vs_xgb:+.6f}")
    print(f"   vs CatBoost: {improvement_vs_catboost:+.6f}")
    print(f"   vs Mejor Individual: {improvement_vs_best_individual:+.6f}")

# 🚀 Aplicar Mejor Estrategia a Test

In [ ]:
def apply_best_strategy_to_test(strategy_name, xgb_test_preds, catboost_test_preds, xgb_acc, catboost_acc):
    """
    Aplicar la mejor estrategia de ensemble a las predicciones de test
    """
    if strategy_name == "XGBoost":
        return xgb_test_preds
    elif strategy_name == "CatBoost":
        return catboost_test_preds
    elif strategy_name == "Simple Voting":
        return simple_voting_ensemble(xgb_test_preds, catboost_test_preds)
    elif strategy_name == "Confidence-based":
        return confidence_based_ensemble(xgb_test_preds, catboost_test_preds, xgb_acc, catboost_acc)
    elif "Weighted XGB=" in strategy_name:
        # Extraer peso del nombre
        weight = float(strategy_name.split("=")[1])
        return weighted_ensemble(xgb_test_preds, catboost_test_preds, weight)
    else:
        print(f"❌ Estrategia no reconocida: {strategy_name}")
        return simple_voting_ensemble(xgb_test_preds, catboost_test_preds)

# Aplicar a predicciones de test
if (xgb_submission is not None and catboost_submission is not None and 
    'best_strategy_name' in locals() and xgb_oof is not None):
    
    print(f"\n🚀 Aplicando estrategia '{best_strategy_name}' a predicciones de test...")
    
    # Obtener predicciones de test
    xgb_test_preds = xgb_submission['Fertilizer Name'].values
    catboost_test_preds = catboost_submission['Fertilizer Name'].values
    
    print(f"📊 Predicciones de test:")
    print(f"  XGBoost: {len(xgb_test_preds):,} predicciones")
    print(f"  CatBoost: {len(catboost_test_preds):,} predicciones")
    
    # Verificar que tengan el mismo tamaño
    if len(xgb_test_preds) != len(catboost_test_preds):
        print("❌ Error: Las predicciones de test tienen diferentes tamaños")
    else:
        # Aplicar mejor estrategia
        ensemble_test_preds = apply_best_strategy_to_test(
            best_strategy_name, 
            xgb_test_preds, 
            catboost_test_preds, 
            xgb_acc, 
            catboost_acc
        )
        
        print(f"✅ Ensemble aplicado exitosamente")
        print(f"   Predicciones generadas: {len(ensemble_test_preds):,}")
        
        # Analizar distribución de predicciones
        print(f"\n📊 Distribución de predicciones ensemble:")
        pred_distribution = pd.Series(ensemble_test_preds).value_counts()
        for fertilizer, count in pred_distribution.head(10).items():
            percentage = count / len(ensemble_test_preds) * 100
            print(f"  {fertilizer}: {count:,} ({percentage:.1f}%)")
        
        # Comparar con distribuciones individuales
        xgb_unique = len(pd.Series(xgb_test_preds).unique())
        catboost_unique = len(pd.Series(catboost_test_preds).unique())
        ensemble_unique = len(pd.Series(ensemble_test_preds).unique())
        
        print(f"\n🎯 Clases únicas predichas:")
        print(f"  XGBoost: {xgb_unique}")
        print(f"  CatBoost: {catboost_unique}")
        print(f"  Ensemble: {ensemble_unique}")
        
        # Analizar concordancia en test
        test_concordance = (xgb_test_preds == catboost_test_preds).mean()
        print(f"\n🤝 Concordancia en test: {test_concordance:.1%}")
        different_test = (xgb_test_preds != catboost_test_preds).sum()
        print(f"🔄 Predicciones diferentes en test: {different_test:,} ({different_test/len(xgb_test_preds):.1%})")

else:
    print("❌ No se pueden aplicar estrategias de ensemble - faltan datos")

# 💾 Guardar Ensemble

In [ ]:
# Guardar resultados del ensemble
if 'ensemble_test_preds' in locals() and 'best_accuracy' in locals():
    
    # Crear nombre del experimento
    ensemble_score_str = f"{best_accuracy:.5f}".replace('.', '')
    ensemble_name = f"Ensemble_XGB-CatBoost_ACC-{ensemble_score_str}"
    ensemble_dir = ENSEMBLE_PATH / ensemble_name
    ensemble_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n💾 Guardando ensemble en: {ensemble_dir}")
    
    # 1. Información del ensemble
    ensemble_info = {
        'ensemble_name': ensemble_name,
        'best_strategy': best_strategy_name,
        'ensemble_accuracy': float(best_accuracy),
        'models_used': {
            'xgb': {
                'model_dir': best_xgb_dir.name,
                'individual_accuracy': float(xgb_acc),
                'map3_score': float(xgb_score)
            },
            'catboost': {
                'model_dir': best_catboost_dir.name,
                'individual_accuracy': float(catboost_acc),
                'map3_score': float(catboost_score)
            }
        },
        'improvements': {
            'vs_xgb': float(improvement_vs_xgb),
            'vs_catboost': float(improvement_vs_catboost),
            'vs_best_individual': float(improvement_vs_best_individual)
        },
        'test_predictions': {
            'total_samples': len(ensemble_test_preds),
            'unique_classes': int(ensemble_unique),
            'concordance_rate': float(test_concordance)
        },
        'created_at': datetime.now().isoformat()
    }
    
    with open(ensemble_dir / f"{ensemble_name}_info.json", 'w') as f:
        json.dump(ensemble_info, f, indent=2)
    
    # 2. Estrategias evaluadas
    strategies_results = {
        'all_strategies': [
            {'name': name, 'accuracy': float(acc)} 
            for name, acc, _ in all_strategies
        ],
        'best_strategy': {
            'name': best_strategy_name,
            'accuracy': float(best_accuracy)
        }
    }
    
    with open(ensemble_dir / f"{ensemble_name}_strategies.json", 'w') as f:
        json.dump(strategies_results, f, indent=2)
    
    # 3. Predicciones OOF del ensemble
    ensemble_oof_df = pd.DataFrame({
        'id': range(len(best_ensemble_preds)),
        'true_label': true_labels,
        'xgb_prediction': xgb_preds,
        'catboost_prediction': catboost_preds,
        'ensemble_prediction': best_ensemble_preds
    })
    
    ensemble_oof_df.to_csv(
        ensemble_dir / f"{ensemble_name}_oof_predictions.csv", 
        index=False
    )
    
    # 4. Submission final
    final_submission = pd.DataFrame({
        'id': range(len(ensemble_test_preds)),
        'Fertilizer Name': ensemble_test_preds
    })
    
    final_submission.to_csv(
        ensemble_dir / f"{ensemble_name}_submission.csv", 
        index=False
    )
    
    # 5. Análisis detallado
    analysis_data = {
        'oof_analysis': {
            'total_samples': len(true_labels),
            'both_correct': int(both_correct.sum()),
            'only_xgb_correct': int(only_xgb_correct.sum()),
            'only_catboost_correct': int(only_catboost_correct.sum()),
            'both_wrong': int(both_wrong.sum()),
            'prediction_concordance': float(pred_concordance),
            'potential_improvement': int(potential_improvement)
        },
        'test_analysis': {
            'total_samples': len(ensemble_test_preds),
            'prediction_concordance': float(test_concordance),
            'different_predictions': int(different_test),
            'ensemble_distribution': pred_distribution.head(10).to_dict()
        }
    }
    
    with open(ensemble_dir / f"{ensemble_name}_analysis.json", 'w') as f:
        json.dump(analysis_data, f, indent=2)
    
    print(f"\n✅ Ensemble guardado exitosamente:")
    print(f"  📁 Directorio: {ensemble_dir}")
    print(f"  ℹ️ Info: {ensemble_name}_info.json")
    print(f"  🎯 Estrategias: {ensemble_name}_strategies.json")
    print(f"  📊 OOF: {ensemble_name}_oof_predictions.csv")
    print(f"  📤 Submission: {ensemble_name}_submission.csv")
    print(f"  🔍 Análisis: {ensemble_name}_analysis.json")

else:
    print("❌ No se puede guardar ensemble - faltan datos")

# 📋 Resumen Final

In [ ]:
if 'ensemble_name' in locals():
    print(f"\n{'='*80}")
    print(f"🏆 RESUMEN FINAL - ENSEMBLE XGBoost + CatBoost")
    print(f"{'='*80}")
    
    print(f"\n🤖 MODELOS UTILIZADOS:")
    print(f"  🚀 XGBoost: {best_xgb_dir.name}")
    print(f"     - MAP@3: {xgb_score:.5f}")
    print(f"     - OOF Accuracy: {xgb_acc:.5f}")
    print(f"     - Estrategia: Variables categóricas codificadas")
    
    print(f"\n  🐱 CatBoost: {best_catboost_dir.name}")
    print(f"     - MAP@3: {catboost_score:.5f}")
    print(f"     - OOF Accuracy: {catboost_acc:.5f}")
    print(f"     - Estrategia: Variables categóricas nativas")
    
    print(f"\n🎯 ENSEMBLE FINAL:")
    print(f"  Nombre: {ensemble_name}")
    print(f"  Estrategia: {best_strategy_name}")
    print(f"  Accuracy: {best_accuracy:.6f}")
    
    print(f"\n📈 MEJORAS LOGRADAS:")
    print(f"  vs XGBoost: {improvement_vs_xgb:+.6f} ({improvement_vs_xgb/xgb_acc*100:+.2f}%)")
    print(f"  vs CatBoost: {improvement_vs_catboost:+.6f} ({improvement_vs_catboost/catboost_acc*100:+.2f}%)")
    print(f"  vs Mejor Individual: {improvement_vs_best_individual:+.6f}")
    
    print(f"\n🤝 DIVERSIDAD DE MODELOS:")
    print(f"  Concordancia OOF: {pred_concordance:.1%}")
    print(f"  Concordancia Test: {test_concordance:.1%}")
    print(f"  Potencial de mejora: {potential_improvement:,} casos")
    
    print(f"\n📊 PREDICCIONES FINALES:")
    print(f"  Total muestras test: {len(ensemble_test_preds):,}")
    print(f"  Clases únicas: {ensemble_unique}")
    print(f"  Diferencias test: {different_test:,} ({different_test/len(ensemble_test_preds):.1%})")
    
    print(f"\n💾 ARCHIVOS GENERADOS:")
    print(f"  📁 {ensemble_dir}")
    print(f"  📤 {ensemble_name}_submission.csv")
    print(f"  📊 {ensemble_name}_oof_predictions.csv")
    print(f"  ℹ️ {ensemble_name}_info.json")
    
    print(f"\n🎉 ENSEMBLE COMPLETADO EXITOSAMENTE!")
    print(f"\n🚀 El archivo de submission está listo para Kaggle.")
    print(f"📈 El ensemble debería superar el rendimiento de ambos modelos individuales.")
    print(f"{'='*80}")

else:
    print("\n❌ No se pudo completar el ensemble")
    print("Verifica que tanto XGBoost como CatBoost estén entrenados correctamente.")